In [1]:
!conda install -c conda-forge folium --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    folium-0.10.0              |             py_1          59 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    branca:          0.3.1-py_0        conda-forge
    folium:          

In [ ]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

wiki_page = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
extract_table = BeautifulSoup(wiki_page,'lxml')
output_table = str(extract_table.table)
temp_df = pd.read_html(output_table)
neighbor_df=temp_df[0]
#neighbor_df.head(10)

temp_df1 = neighbor_df[neighbor_df.Borough != 'Not assigned']

temp_df2 = temp_df1.groupby(['Postcode','Borough'], sort=False).agg(', '.join)
temp_df2.reset_index(inplace=True)

temp_df2['Neighbourhood'] = np.where(temp_df2['Neighbourhood'] == 'Not assigned',temp_df2['Borough'], temp_df2['Neighbourhood'])
#temp_df2.head(10)
#temp_df2.shape

location_info = pd.read_csv('https://cocl.us/Geospatial_data', header=0)
location_info.rename(columns={'Postal Code':'Postcode'}, inplace=True)

final_df=pd.merge(temp_df2,location_info, on='Postcode')

final_df.head(10)

Create another dataframe where each row contains Toronto in the "Borough" column.

In [ ]:
only_toronto_borough = final_df[final_df['Borough'].str.contains('Toronto',regex=False)]
only_toronto_borough.head()

apply KNN clustering to the a new dataframe "five_clusters" with 5 clusters. Since Postcode, Borough, and Neighborhood are not numbers, we drop them in the new dataframe, and only the location longitude and latitude data are sufficient for clsutering purpose. After clustering, we insert the cluster labels to the original dataframe "only_toronto_borough".

In [ ]:
from sklearn.cluster import KMeans
k=5
five_clusters = only_toronto_borough.drop(['Postcode','Borough','Neighbourhood'],1)
kmeans = KMeans(n_clusters = k,random_state=0).fit(five_clusters)
only_toronto_borough.insert(0, 'Cluster Labels', kmeans.labels_)

view the five clusters on the map

In [ ]:
import folium
import matplotlib.colors as colors
import matplotlib.cm as cm
# create map
map_clusters = folium.Map(location=[43.651070,-79.347015],zoom_start=12)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, neighbourhood, cluster in zip(only_toronto_borough['Latitude'], only_toronto_borough['Longitude'], only_toronto_borough['Neighbourhood'], only_toronto_borough['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters